In [14]:
!pip3 install matplotlib shapely dubins pyyaml descartes

In [15]:
from __future__ import division

%load_ext autoreload
%autoreload 2
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from shapely.geometry import Point, Polygon, LineString, box
from environment import Environment, plot_environment, plot_line, plot_poly

import dubins

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Motion Planning with Dynamic Constraints

The mad bomber from the 1994 movie Speed (https://www.youtube.com/watch?v=8piqd2BWeGI) strikes again! There's a bomb under the commuter bus that will explode if the bus goes under 50 mph. In order to detonate the bomb, the bus has to meet the bomb squad at the airport without stopping or hitting any obstacles on the way. Luckily, the bus happens to be an autonomous vehicle capable of performing motion planning.

We'll be solving this problem using the following methods: A* search, Constrained RRT and RRT*.


## Map and Obstacle Constraints
![map_motion](map.PNG)

Below is code that creates an easy and hard environment of Los Angeles, where the purple polygons represents landmarks/buildings. The green polygon represents the airport.

In [ ]:
def random_environment(bounds, start, radius, goal, n, size_limits=(0.5, 1.5)):
    minx, miny, maxx, maxy = bounds
    print(bounds)
    edges = 4
    minl, maxl = size_limits
    env = Environment(None)
    obs = []
    start_pose = Point(start).buffer(radius, resolution=3)
    obi = 0
    while obi < n:
        r = np.random.uniform(low=0.0, high=1.0, size=2)
        xy = np.array([minx + (maxx-minx)*r[0], miny + (maxy-miny)*r[1]])
        
        angles = np.random.rand(edges)
        angles = angles*2*np.pi / np.sum(angles)
        for i in range(1,len(angles)):
            angles[i] = angles[i-1] + angles[i]
        angles = 2*np.pi * angles / angles[-1] 
        angles = angles + 2*np.pi*np.random.rand()
        lengths = 0.5*minl + (maxl-minl) * 0.5 * np.random.rand(edges)
        xx = xy[0] + np.array([l*np.cos(a) for a,l in zip(angles,lengths)])
        yy = xy[1] + np.array([l*np.sin(a) for a,l in zip(angles,lengths)])
        p = Polygon([(x,y) for x,y in zip(xx,yy)])
        if p.intersects(start_pose) or p.intersects(goal):
            continue
        else:
            obi = obi + 1
            obs.append(p)
#         coords = xy + [l*np.cos(a),l*np.sin(a) for a,l in zip(angles,lengths)]
    env.add_obstacles(obs)
    return env
start = (-4,-2)
radius = 0.1
goal_region = Polygon([(12,3), (12,4), (13,4),(13,3)])
bounds = (-5, -4, 15, 5)
env = random_environment(bounds, start, radius, goal_region, 30, (0.2, 0.4))
ax = plot_environment(env)
plot_poly(ax, goal_region,'green')


start = (-4,-2)
radius = 0.1
goal_region = Polygon([(12,3), (12,4), (13,4),(13,3)])
bounds = (-5, -4, 15, 5)
env = random_environment(bounds, start, radius, goal_region, 80, (0.2, 0.4))
ax = plot_environment(env)
plot_poly(ax, goal_region,'green')


## Bus Motion Constraints

![bus_motion](car_motion.png)

Because the car has wheels that steer in either direction, we constrain its motion to arcs, specifically a Dubins path. Given the direction of the bus (in the image above, it's facing north), we can parametrize the range of its motion with $D$ degrees, which is the arc that it can forward in, and $R$, the turning radius. The state of the car will be represented as a tuple of length 3: (x coordinate, y coordinate, pose/direction).

In the image above, there are three different examples of the bus at possible states. When the bus is at the bottom-most state, the green area represents feasible states it can move to, and the red areas represent states that it cannot move to.

## Planning with Motion Primitives

One way to solve our motion planning problem is search. In order to take into consideration our constraints, we need to be careful about how we discretize our search space. A naive approach would simply divide the free configuration space into a grid. However, when we search this grid, we might create an infeasable path. The bus cannot drive backwards due to its constant speed. Similarly, the bus cannot reach the cell directly to the left and right. 

A better discretization method would be to select from a precomputed list of "motion primitives." These are a list of actions from any state to a set of reachable states. This creates a tree with a root in  the nodes are reachable A larger number of motion primitives result in larger branching factors of the search tree because

## RRT with Kinematic Constraints

Discretizing the range of motion for the bus creates a tree with exponential size, and unfortunately, the bomb will detonate sooner than expected. Therefore, we need an algorithm that can plan a path to the airport, even if suboptimal, in a shorter amount of time. Instead of splitting the sample space into intervals, we consider Rapidly-exploring Random Trees (RRT), to explore the space more efficiently.

However, the main issue here is that if we sample a point anywhere in the environment, there is a chance that it is not reachable from the car's current location and pose via a linear path. Also, due to the bus's limited range of motion, we want to accomodate for those kinematics. Therefore, we propose the following pseudocode, which modifies the linear nature of traditional RRT:

- Sample randomly from the enviroment
- Find the closest point in the tree to the sampled point
- Draw a dublin path between the randomly sampled point and the neighboring point
    - If it hits an obstacle, then we resample
- If there's no obstacle in the way, then we add the sampled point to the tree
    - We will have a max length $L$, so we will cut the path off at $L$ if it is longer than $L$, and add the point at which the path is cut off to the tree instead
    - Note: the dublin path will essentially be in a 3D space because we have three parameters (x, y, pose)


In [ ]:
def check_obstacle():
    
def RRT():
    

## Goal-directed Sampling

Since the autonomous bus has information about the area and knows where the goal state is, we implement a heuristic in the random sampling to find the airport faster. Essentially, with probability $p$, the algorithm will explore a new offshoot of the tree by randomly sampling. With probability $1-p$, the algorithm will sample in the direction and general location of the goal state. This way, we can exploit knowing the goal state while also exploring other parts of the environment.

In [ ]:
def RRT_goal_directed():
    p = 0.7 # p = 1 would be the same as the normal RRT implemented above
    
    if random() > p:
        # Algorithm samples towards the goal
    else:
        # Standard RRR
    

## RRT*

RRT* is an optimized version of RRT. The main difference is that every time we obtain a new point, we must check to see whether the paths of other nodes can be reduced by going through the new point. If so, we have to update our graph. Like RRT, we will be using the dubin library for path generation. 

In [ ]:
def RRT_star(N):
    """
    Runs like RRT except it considers minimum cost paths and changes tree structure
    """
    
    V = dict()
    E = set()
    for i in range(N):
        # x_rand = random point
        # use dubins to generate path between x_rand and bus
        
        # connect along minimum cost path
        
        # rewire the tree if paths with lower costs exist
        

In [18]:

q0 = (1, 2, 1)
q1 = (3, 4, 2)
turning_radius = 1.0
step_size = 0.5

path = dubins.shortest_path(q0, q1, turning_radius)
configurations, _ = path.sample_many(step_size)
configurations

[(1.0, 2.0, 1.0),
 (1.3620454462036935, 2.337280256022233, 0.5),
 (1.8365771040969232, 2.4893460489960906, 0.2453562404811206),
 (2.3216025316138618, 2.6107970047848816, 0.2453562404811206),
 (2.759792644254954, 2.840671841848695, 0.7329837415798964),
 (3.0341844572793333, 3.252429062992962, 1.2329837415798965),
 (3.077578999993752, 3.745330462749405, 1.7329837415798965)]